In this notebook, we'll build a model to classify online posts about baseball and hockey.

Below we download the online posts data.

In [1]:
from sklearn.datasets import fetch_20newsgroups
# from sklearn.feature_extraction.text import TfidfVectorizer
groups = ['rec.sport.baseball', 'rec.sport.hockey']
newsgroups = fetch_20newsgroups(subset='all', remove = ['headers', 'footers', 'quotes'], categories = groups)

In [2]:
# from sklearn.naive_bayes import MultinomialNB
# from sklearn import metrics
# newsgroups_test = fetch_20newsgroups(subset='test', remove = ['headers', 'footers', 'quotes'], categories = groups)
# vectors_test = vectorizer.transform(newsgroups_test.data)
# clf = MultinomialNB(alpha=.01)
# clf.fit(vectors, newsgroups_train.target)
# pred = clf.predict(vectors_test)
# metrics.f1_score(newsgroups_test.target, pred, average='macro')
# 0.88213592402729568 (full set)
# 0.9320767597087378

Next, we download GloVe vectors we will be using to represent our post data.

In [3]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-07-17 19:05:59--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-07-17 19:05:59--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-07-17 19:06:00--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

Below we unzip the GloVe file we downloaded.

In [4]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


Next, we load the GloVe vectors.

In [5]:
import numpy as np

embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


Next, we convert the data to a collection of word GloVe word vectors for each of the words in our dataset.

In [6]:
!pip install keras=='2.3.1'
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

print('Preparing embedding matrix.')
MAX_NUM_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(newsgroups.data)
sequences = tokenizer.texts_to_sequences(newsgroups.data)

word_index = tokenizer.word_index

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Using TensorFlow backend.


Preparing embedding matrix.


Next, we'll build our dataset for training, `data` and `labels`, as well as our test set, `data_test` and `labels_test`.  We will limit our training set to 200 examples.

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# finally, vectorize the text samples into a 2D integer tensor
MAX_NUM_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(newsgroups.data)
sequences = tokenizer.texts_to_sequences(newsgroups.data)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(newsgroups.target))

# print(data.shape)

data, data_test, labels, labels_test = train_test_split(data,labels,train_size=200)

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
print('Shape of data_test tensor:', data_test.shape)
print('Shape of label_test tensor:', labels_test.shape)

# split the data into a training set and a validation set
# indices = np.arange(data.shape[0])
# np.random.shuffle(indices)
# data = data[indices]
# labels = labels[indices]
# num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

# x_train = data[:-num_validation_samples]
# y_train = labels[:-num_validation_samples]
# x_val = data[-num_validation_samples:]
# y_val = labels[-num_validation_samples:]

Found 18135 unique tokens.
Shape of data tensor: (200, 1000)
Shape of label tensor: (200, 2)
Shape of data_test tensor: (1793, 1000)
Shape of label_test tensor: (1793, 2)


Next, we'll declare a `train` function that declares and trains the model with `pretrain` weights.  

In [8]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant
# from keras.optimizers import RMSprop
# from keras.optimizers import Adam
from keras import optimizers

EMBEDDING_DIM = 100

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
# num_words = len(vectorizer.vocabulary_)
# num_words = len(word_index)+1

def train(pretrain):
  if not pretrain:  # train your own embedding
    embedding_layer = Embedding(num_words,
                              EMBEDDING_DIM,
                              input_length=MAX_SEQUENCE_LENGTH,
                              trainable=True
                             )
  else:
      embedding_layer = Embedding(num_words,
                              EMBEDDING_DIM,
                              embeddings_initializer=Constant(embedding_matrix),
                              input_length=MAX_SEQUENCE_LENGTH,
                              trainable=False
                           )
  print('Training model.')

  # train a 1D convnet with global maxpooling
  sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

  embedded_sequences = embedding_layer(sequence_input)
  x = Conv1D(128, 5, activation='relu')(embedded_sequences)
  x = MaxPooling1D(5)(x)
  x = Conv1D(128, 5, activation='relu')(x)
  x = MaxPooling1D(5)(x)
  x = Conv1D(128, 5, activation='relu')(x)
  x = GlobalMaxPooling1D()(x)
  x = Dense(128, activation='relu')(x)
  preds = Dense(len(groups), activation='softmax')(x)

  solver = optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False)

  model = Model(sequence_input, preds)
  model.compile(loss='categorical_crossentropy',
                optimizer=solver,
                metrics=['acc'])

  model.fit(data, labels,
            epochs=100,
            validation_data=(data_test, labels_test))

Below we train the model without pretrained weights.

In [9]:
train(False)

Training model.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 200 samples, validate on 1793 samples
Epoch 1/100
200/200 [==============================] - 8s 38ms/step - loss: 0.6949 - acc: 0.5150 - val_loss: 0.6943 - val_acc: 0.4964
Epoch 2/100
200/200 [==============================] - 0s 1ms/step - loss: 0.6807 - acc: 0.5450 - val_loss: 0.6951 - val_acc: 0.4964
Epoch 3/100
200/200 [==============================] - 0s 1ms/step - loss: 0.6680 - acc: 0.5450 - val_loss: 0.6963 - val_acc: 0.4964
Epoch 4/100
200/200 [==============================] - 0s 1ms/step - loss: 0.6534 - acc: 0.5450 - val_loss: 0.6987 - val_acc: 0.4964
Epoch 5/100
200/200 [==============================] - 0s 1ms/step - loss: 0.6319 - acc: 0.5800 - val_loss: 0.6940 - val_acc: 0.5148
Epoch 6/100
200/200 [==============================] - 0s 1ms/step - loss: 0.5998 - acc: 0.7550 - val_loss: 0.7023 - val_acc: 0.5125
Epoch 7/100
200/200 [==============================] - 0s 1ms/step - loss: 0.5474 - acc: 0.7500 - val_loss: 0.6755 - val_acc: 0.5700
Epoch 8/100
200/200 [

Next we train the model with pretrained weights.

In [10]:
train(True)

Training model.
Train on 200 samples, validate on 1793 samples
Epoch 1/100
200/200 [==============================] - 0s 2ms/step - loss: 0.7112 - acc: 0.5600 - val_loss: 0.6971 - val_acc: 0.5159
Epoch 2/100
200/200 [==============================] - 0s 1ms/step - loss: 0.6341 - acc: 0.6800 - val_loss: 0.6797 - val_acc: 0.5477
Epoch 3/100
200/200 [==============================] - 0s 1ms/step - loss: 0.5904 - acc: 0.7850 - val_loss: 0.6803 - val_acc: 0.5427
Epoch 4/100
200/200 [==============================] - 0s 1ms/step - loss: 0.5447 - acc: 0.8400 - val_loss: 0.7295 - val_acc: 0.4992
Epoch 5/100
200/200 [==============================] - 0s 1ms/step - loss: 0.4859 - acc: 0.8100 - val_loss: 0.6293 - val_acc: 0.6793
Epoch 6/100
200/200 [==============================] - 0s 1ms/step - loss: 0.4065 - acc: 0.9400 - val_loss: 0.6045 - val_acc: 0.6899
Epoch 7/100
200/200 [==============================] - 0s 1ms/step - loss: 0.3392 - acc: 0.9450 - val_loss: 0.5767 - val_acc: 0.7072
Epoch 